In [1]:
# 基本モジュールのインポート
import sys, requests, datetime, lxml, os
from itertools import repeat
from datetime import timedelta

# 日付を簡易的に変更するモジュール
# https://qiita.com/xza/items/9618e25a8cb08c44cdb0
from dateutil.relativedelta import relativedelta

# ウェブスクレイピング用モジュール
# https://qiita.com/itkr/items/513318a9b5b92bd56185
from lxml.html import fromstring
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 並列処理を行う上で必要なモジュール
# https://docs.python.org/ja/3/library/multiprocessing.html
from multiprocessing import Process
from concurrent.futures import ProcessPoolExecutor

# データフレーム格納処理に使用するモジュール
import numpy as np
import pandas as pd

# 個人で作成したスクリプト：関数やコードのリストなど
sys.path.insert(0, './src/')

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
!date

Wed Jul  1 03:27:29 JST 2020


### 1. 地方競馬場コードの定義

データ取得先：**'https://www.jbis.or.jp/'**

地方競馬場のレース結果と出場競走馬を取得するため、上記ウェブサイト上で定義されている各競馬場のコードを定義する。


In [3]:
chihou_keibajo = {
    "岩見沢": '205', "帯広": '206', "旭川": '207', 
    "札幌": '208', "函館": '209', "門別": '236', 
    "水沢": '211', "盛岡": '210', "上山": '212', 
    "新潟": '213', "三条": '214', "足利": '215',
    "宇都宮": '216', "高崎": '217', "浦和": '218',
    "船橋": '219', "大井": '220', "川崎": '221',
    "金沢": '222', "笠松": '223', "名古屋": '224',
    "中京": '225', "園田": '227', "姫路": '228',
    "益田": '229', "福山": '230', "高知": '231',
    "佐賀": '232', "荒尾": '233', "中津": '234' 
}

### 2. 関数の定義

複数回使用するコードが存在するため、それを関数とする

In [4]:
def validate_dir(path, mkdir=True):
    '''ディレクトリが存在するかを確認する'''

    if not os.path.isdir(path):
        
        # mkdirパラメータがtrueの場合、ディレクトリを作成する
        if mkdir:
            os.mkdir(path)
            #print('{} created.'.format(path))
        else:
            return False
        
    return True
    
def get_tables(session, url):
    '''ページソースからテーブルを取得する'''
    
    try:
        response = session.get(url)
        soup = bs(response.content)
        tables = soup.find_all('table')
    except Exception as e:
        # 現在はロガーを定義していないのでprintで返す
        print("{}".format(e))
        return None
    
    return tables

In [5]:
# マルチプロセスを使用するための関数の定義
def run(r_loc, h_loc, baseURL, url):
    session = requests.Session()
    tables = get_tables(session, url)
    
    # レース結果を取得し、保存する
    f_name = url.split('/')[-2] + '.csv'
    pd.read_html(str(tables))[1].to_csv(r_loc + '/' + f_name, index=False)
    
    # レースに出場した競走馬の情報のURLを取得する
    h_hrefs = [baseURL + a['href'] for a in tables[1].find_all('a', href=True) if a['href'].find('horse') != -1]
    
    dfs = []
    
    # 各馬毎の基本情報を取得する
    for hurl in h_hrefs:

        h_desc = session.get(hurl)
        h_soup = bs(h_desc.content)
        h_tables = h_soup.find_all('table')
        
        # 競走馬情報のテーブルを作成する
        cols = ['番号', '競走馬']
        data = []
        
        # 番号をURLから取得
        data.append(hurl.split('/')[-2])
        
        # 競走馬名を取得
        data.append(h_soup.find('h1', {'class':'hdg-l1-02'}).text)
        
        # 競走馬の属性を取得
        for th, td in zip(h_tables[0].find_all('th'), h_tables[0].find_all('td')):
            cols.append(th.text)
            data.append(td.text)
        
        # 競走馬の通算成績を取得(失敗した場合は、属性のみをリストに格納する)
        try:
            h_res = pd.read_html(str(h_tables))[2]
            h_res.rename({'Unnamed: 0': '種類'}, axis=1, inplace=True)
            
            # 属性情報に結合する
            dfs.append(pd.concat([pd.DataFrame([data], columns=cols),
                                  pd.DataFrame([h_res[h_res['年'] == '合計'].values.ravel()])], axis=1))
        except:
            dfs.append(pd.DataFrame([data], columns=cols))
            
    # 取得したデータフレームのリストをすべて結合し、保存する
    h_fname = url.split('/')[-4] + '_' + url.split('/')[-3] + '_' + url.split('/')[-2] + '.csv'
    pd.concat(dfs, axis=0).to_csv(h_loc + '/' + h_fname, index=False, encoding='utf-8')

### 3. スクレイピング開始前設定

    1.データ抽出開始年月の設定
    2.データ抽出先ベースURLの設定
    3.抽出データ格納先の設定
    4.セッションを作成

In [6]:
# データ抽出開始年月の設定(2020年6月時点での過去3年分のデータを取得)
date_counter = datetime.date(2017, 8, 1)
date_counter.strftime('%Y%m%d')

'20170801'

In [7]:
# データ抽出先ベースURL
baseURL = 'https://www.jbis.or.jp'
baseURL

'https://www.jbis.or.jp'

In [8]:
# 抽出データ格納先の設定
# r_dir = レース結果格納先, h_dir = 競走馬情報の格納先
r_dir = './data/r_res/'
h_dir = './data/h_desc/'

validate_dir(r_dir), validate_dir(h_dir)

(True, True)

In [9]:
# セッションのインスタンス化
session = requests.Session()
session

In [10]:
# マルチプロセスクラスのインスタンス化
n_proc = 24
executor = ProcessPoolExecutor(max_workers=n_proc)

### 4. スクレイピング

データ取得先：'https://www.jbis.or.jp/race/calendar/[YYYYMM]/'

In [12]:
while not date_counter.strftime('%Y%m') == '202007':
    
    # 月日の取得
    ym = date_counter.strftime('%Y%m')
    
    # ソース取得先URLの定義(ベースURL+'/race/calendar'+月日)
    calendarURL = baseURL + '/race/calendar/' + ym + '/'
    
    print(calendarURL)
    
    # 取得先からページソースを取得する
    try:
        response = session.get(calendarURL)
        c_soup = bs(response.content)
    except Exception as e:
        print('{}'.format(e))
        
        # 取得に失敗し場合、次のループに飛ぶ
        date_counter = date_counter + relativedelta(months=1)
        continue
        
    # ページソースから日付毎に各競技場のレース結果のURLのリストを取得する
    c_hrefs = [baseURL + a['href'] for a in c_soup.find('tbody').find_all('a', href=True)]
    
    # 取得したURLリストをループする
    for href in c_hrefs:
        
        # テーブル取得
        tables = get_tables(session, href)
        
        # 保存先ディレクトリが存在しない場合、作成する
        r_loc_dir = r_dir + href.split('/')[-2] + '/'
        validate_dir(r_loc_dir)
        
        r_loc_dir = r_loc_dir + href.split('/')[-3] + '/'
        validate_dir(r_loc_dir)
        
        # pandasのread_htmlを使用してデータフレームを作成し、保存する
        try:
            f_name = href.split('/')[-3] + '.csv'
            pd.read_html(str(tables))[0].to_csv(r_loc_dir + f_name, index=False, encoding='utf-8')
    
            # 競技場で行われた各レースごとの詳細結果があるURLを取得する
            race_res_hrefs = [baseURL + a['href'] for a in tables[0].find_all('a', href=True) if a['href'].find('race') != -1]
        
            # 取得したURLごとにマルチプロセスでスクレイピングを行う
            executor.map(run, 
                         repeat(os.path.abspath(r_loc_dir)),
                         repeat(os.path.abspath(h_dir)),
                         repeat(baseURL),
                         race_res_hrefs)
        except:
            continue
        
        #for rhref in race_res_hrefs:
        #    tables = get_tables(session, rhref)
        #
        #    # レース結果を取得し、保存する
        #    f_name = rhref.split('/')[-2] + '.csv'
        #    pd.read_html(str(tables))[1].to_csv(r_loc_dir + f_name, index=False)
        #    
        #    # レースに出場した競走馬の情報のURLを取得する
        #    h_hrefs = [a['href'] for a in tables[1].find_all('a', href=True) if a['href'].find('horse') != -1]
        #    
        #    dfs = []
        #    
        #    # 各馬毎の基本情報を取得する
        #    for hurl in h_hrefs:
        #
        #        h_desc = session.get(baseURL + hurl)
        #        h_soup = bs(h_desc.content)
        #        h_tables = h_soup.find_all('table')
        #        
        #        # 競走馬情報のテーブルを作成する
        #        cols = ['番号', '競走馬']
        #        data = []
        #        
        #        # 番号をURLから取得
        #        data.append(hurl.split('/')[-2])
        #        
        #        # 競走馬名を取得
        #        data.append(h_soup.find('h1', {'class':'hdg-l1-02'}).text)
        #        
        #        # 競走馬の属性を取得
        #        for th, td in zip(h_tables[0].find_all('th'), h_tables[0].find_all('td')):
        #            cols.append(th.text)
        #            data.append(td.text)
        #        
        #        # 競走馬の通算成績を取得(失敗した場合は、属性のみをリストに格納する)
        #        try:
        #            h_res = pd.read_html(str(h_tables))[2]
        #            h_res.rename({'Unnamed: 0': '種類'}, axis=1, inplace=True)
        #            
        #            # 属性情報に結合する
        #            dfs.append(pd.concat([pd.DataFrame([data], columns=cols),
        #                                  pd.DataFrame([h_res[h_res['年'] == '合計'].values.ravel()])], axis=1))
        #        except:
        #            dfs.append(pd.DataFrame([data], columns=cols))
        #            
        #    # 取得したデータフレームのリストをすべて結合し、保存する
        #    h_fname = rhref.split('/')[-4] + '_' + rhref.split('/')[-3] + '_' + rhref.split('/')[-2] + '.csv'
        #    pd.concat(dfs, axis=0, ignore_index=True).to_csv(h_dir + h_fname, index=False, encoding='utf-8')
    
    # 1カ月先に進める
    date_counter = date_counter + relativedelta(months=1)

https://www.jbis.or.jp/race/calendar/201708/
https://www.jbis.or.jp/race/calendar/201709/
https://www.jbis.or.jp/race/calendar/201710/
https://www.jbis.or.jp/race/calendar/201711/
https://www.jbis.or.jp/race/calendar/201712/
https://www.jbis.or.jp/race/calendar/201801/
https://www.jbis.or.jp/race/calendar/201802/
https://www.jbis.or.jp/race/calendar/201803/
https://www.jbis.or.jp/race/calendar/201804/
https://www.jbis.or.jp/race/calendar/201805/
https://www.jbis.or.jp/race/calendar/201806/
https://www.jbis.or.jp/race/calendar/201807/
https://www.jbis.or.jp/race/calendar/201808/
https://www.jbis.or.jp/race/calendar/201809/
https://www.jbis.or.jp/race/calendar/201810/
https://www.jbis.or.jp/race/calendar/201811/
https://www.jbis.or.jp/race/calendar/201812/
https://www.jbis.or.jp/race/calendar/201901/
https://www.jbis.or.jp/race/calendar/201902/
https://www.jbis.or.jp/race/calendar/201903/
https://www.jbis.or.jp/race/calendar/201904/
https://www.jbis.or.jp/race/calendar/201905/
https://ww

In [13]:
!pip3 install html5lib

    100% |████████████████████████████████| 112kB 1.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f4/24/2a3e3df732393fed8b3ebf2ec078f05546de641fe1b667ee316ec1dcf3b7/webencodings-0.5.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
